In [1]:
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer

# Load HTML, gpt researcher
url = "https://klientboost.com/"

In [2]:
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()


True

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import pprint
import os
import asyncio
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import AsyncHtmlLoader, AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer
from langchain.chains import create_extraction_chain
from langchain.chat_models import ChatOpenAI

async def scrape_with_playwright(urls, schema):
    loader = AsyncHtmlLoader(urls)
    docs = loader.load()  # Make sure to use await for an async function
    
    bs_transformer = BeautifulSoupTransformer()
    docs_transformed = bs_transformer.transform_documents(docs)
    print("Extracting content with LLM")

    # Grab the first 1000 tokens of the site
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=50, chunk_overlap=0)
    splits = splitter.split_documents(docs_transformed)

    print(splits[0].page_content)

    llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
    chain = create_extraction_chain(schema=schema, llm=llm)
    #extracted_content = chain.run(splits[0].page_content)
    #get only most probable answer
    extracted_content = chain.run(splits[0].page_content)[0]
    return extracted_content

schema = {
    "properties": {
        "hero_header": {"type": "string"},
    },
    "required": ["hero_header"],
}

# Define your URLs here
urls = ["https://abtesting.ai/"]
loop = asyncio.get_event_loop()

# Run the async function within the Jupyter event loop
await scrape_with_playwright(urls, schema)

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 14.16it/s]


Extracting content with LLM
The easiest, fastest and most efficient way to A/B test. Insert your URL below and start testing now. Thanks to the use of GPT-3, we offer automated text suggestions for your headline, copy and Call To Action. Save time


{'hero_header': 'The easiest, fastest and most efficient way to A/B test.'}

In [37]:
import requests
from bs4 import BeautifulSoup
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
        }
url = "https://klientboost.com/"
req = requests.get(url,headers=headers)
soup = BeautifulSoup(req.content, "html.parser")

# get all p and header and text size, sort by text size and location
elements = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"])
def get_font_size(element):
    style = element.get("style", "")
    style_parts = style.split(";")
    for part in style_parts:
        if "font-size" in part:
            size_parts = part.split(":")
            if len(size_parts) == 2:
                return float(size_parts[1].strip().replace("px", "").replace("pt", ""))
    return None

sorted_elements = sorted(elements, key=lambda element: get_font_size(element) or 0)

# Print the sorted elements
for element in sorted_elements:
    font_size = get_font_size(element)
    if font_size is not None:
        print(f"Font Size: {font_size} - {element.text.strip()}")
    else:
        print(f"Font Size not found - {element.text.strip()}")






Font Size not found - The Performance Marketing Agency
That Doubles Revenue, Not Your Budget
Font Size not found - Times are tough right now. Work with a marketing agency 
that drives more pipeline, revenue, and profit without sacrificing quality.
Font Size not found - Winning marketing data from 250+ active clients, customized for you
Font Size not found - Winning marketing data from 250+ active clients, customized for you
Font Size not found - One Agency + Multiple Marketing Channels = 
More Money In Your Pocket
Font Size not found - We help companies scale their strategies across multiple channels 
to drive more revenue, more quickly, without cutting corners.
Font Size not found - Your Custom Marketing Plan Shows 
The Exact Steps We’ll Take To Hit Your Goals
Font Size not found - Get years of data-driven experimentation, through hundreds of millions of marketing spend,
across thousands of companies, absolutely free.
Font Size not found - Your Marketing Will Perform Better With Us
Fo

In [47]:
def score_element(element, max_index, max_font_size, index):
    """
    Score an element based on its position, type, and font size.
    
    Args:
        element (Tag): The BeautifulSoup Tag representing the HTML element.
        max_index (int): The maximum index (position) of an element in the list.
        max_font_size (float): The maximum font size found among elements.

    Returns:
        float: The score for the element, where higher scores indicate a higher probability of being the hero header.
    """
    element_type = element.name
    font_size = get_font_size(element)

    # Weight factors for scoring (you can adjust these as needed)
    position_weight = 1.0  # Weight for element position (higher is more important)
    type_weight = 0.5  # Weight for element type (higher is more important)
    font_size_weight = 0.3  # Weight for font size (higher is more important)

    # Calculate scores for position, type, and font size
    position_score = 1.0 - (index / max_index)  # Higher position means lower score
    type_score = 1.0 if element_type in ["h1", "h2", "h3"] else 0.0  # Headers get higher score
    font_size_score = font_size / max_font_size if font_size is not None else 0.0

    # Calculate the final score by combining weighted scores
    final_score = (
        (position_weight * position_score)
        + (type_weight * type_score)
        + (font_size_weight * font_size_score)
    )

    return final_score



In [1]:
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
        }
url = "https://buzzbonus.tech"
req = requests.get(url,headers=headers)
soup = BeautifulSoup(req.content, "html.parser")

NameError: name 'requests' is not defined

In [54]:
elements = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"])

# Determine the maximum index and maximum font size among elements
max_index = len(elements) - 1
max_font_size = max(get_font_size(element) or 0.0 for element in elements)

# Initialize variables to track the element with the highest score
highest_score = None
best_element = None
best_selector = None

index_elements = {
    "h1": 0,
    "h2": 0,
    "h3": 0,
    "h4": 0,
    "h5": 0,
    "h6": 0,
    "p": 0,
}

# Score elements and find the highest-scoring element
for index, element in enumerate(elements):
    element_score = score_element(element, max_index, max_font_size, index)
    
    # Check if the current element has a higher score than the previous best
    if highest_score is None or element_score > highest_score:
        highest_score = element_score
        best_element = element
        best_selector = (best_element.name, best_element.attrs, best_element.text, index_elements[best_element.name])
    index_elements[element.name] += 1

# Print the highest-scoring element and its selector
if best_element is not None:
    print(f"Highest Scoring Element (Score: {highest_score:.2f}):")
    print(f"Selector: {best_selector}")
    print(f"Text: {best_element.text.strip()}")
else:
    print("No elements found.")

Highest Scoring Element (Score: 1.50):
Selector: ('h1', {'class': ['text-4xl', 'md:text-6xl', 'font-extrabold', 'text-center']}, 'SuperchargeYour Online Presence', 0)
Text: SuperchargeYour Online Presence


In [43]:
import requests
from bs4 import BeautifulSoup

# Define the URL you want to scrape


# Define user-agent headers
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
}

# Send an HTTP request to the URL and parse the HTML content
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")

# Find all header (h1, h2, h3, etc.) and paragraph (p) elements
elements = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"])

# Create a function to extract font size from a style attribute
def get_font_size(element):
    style = element.get("style", "")
    style_parts = style.split(";")
    for part in style_parts:
        if "font-size" in part:
            size_parts = part.split(":")
            if len(size_parts) == 2:
                return float(size_parts[1].strip().replace("px", "").replace("pt", ""))
    return None

# Create a list to store tuples (element type, text, font size, selector)
elements_info = []

# Extract information and create tuples for header elements
for element in elements:
    element_type = element.name
    text = element.text.strip()
    font_size = get_font_size(element)
    elements_info.append((element_type, text, font_size))

# Sort the tuples by font size (in ascending order)
# elements_info.sort(key=lambda x: x[2] if x[2] is not None else 0)


# Print the sorted tuples
for element_info in elements_info:
    element_type, text, font_size = element_info
    print(f"Type: {element_type}, Text: {text}, Font Size: {font_size}")


Type: h1, Text: The Performance Marketing Agency
That Doubles Revenue, Not Your Budget, Font Size: None
Type: h2, Text: Times are tough right now. Work with a marketing agency 
that drives more pipeline, revenue, and profit without sacrificing quality., Font Size: None
Type: h3, Text: Winning marketing data from 250+ active clients, customized for you, Font Size: None
Type: h3, Text: Winning marketing data from 250+ active clients, customized for you, Font Size: None
Type: h2, Text: One Agency + Multiple Marketing Channels = 
More Money In Your Pocket, Font Size: None
Type: h3, Text: We help companies scale their strategies across multiple channels 
to drive more revenue, more quickly, without cutting corners., Font Size: None
Type: h2, Text: Your Custom Marketing Plan Shows 
The Exact Steps We’ll Take To Hit Your Goals, Font Size: None
Type: h3, Text: Get years of data-driven experimentation, through hundreds of millions of marketing spend,
across thousands of companies, absolutely fr